In [29]:
import tkinter as tk


In [30]:

#bluetooth
def discovery():
    import bluetooth
    import csv
    index = 1
    nearby_devices = bluetooth.discover_devices(lookup_names=True)
    #print("found %d devices" % len(nearby_devices))

    # Define the filename for the CSV file
    filename = 'Bluetooth/Known_users.csv'
    with open(filename, mode='r') as csv_file:
        content = csv.reader(filename)
        for addr, name in nearby_devices:
            if nearby_devices[0][0] == addr and nearby_devices[0][1] == name:
                thing = "Welcome "+name
                label.configure(text = thing)
            else:
                with open(filename, mode='w',newline='') as csv_file:
                    # Create a writer object
                    writer = csv.writer(csv_file)
                    for addr, name in nearby_devices:
                        # Write the data to the CSV file
                        writer.writerow([addr,name])
                        index = index + 1
                        #print(" %s - %s" % (addr, name))
                        thing = "new user detected"
                        label.configure(text = thing)
    

In [28]:

# Create the main window
root = tk.Tk()

root.after(0, discovery)
# Set the title and size of the window
root.title("ADHD SUPPORT SYSTEM AAAAAAA!!!")
root.geometry("400x300")
# Add a label to the window
label = tk.Label(root, text="Hello, World!")
label.pack()
# Add a button to the window
#bluetooth_button = tk.Button(root, text="Bluetooth discovery",command=discovery)
#bluetooth_button.pack()
# Add a button to the window
button = tk.Button(root, text="Click me!")
button.pack()
# Add a button to the window
button = tk.Button(root, text="Click me!")
button.pack()
# Add a button to the window
button = tk.Button(root, text="Click me!")
button.pack()
# Run the main loop
root.mainloop()